In [2]:
import pandas as pd
import sqlite3

In [3]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
connector = sqlite3.connect("restaurants.db")

with open("restaurants.sql","r") as f:
    sql_script = f.read()

connector.executescript(sql_script)

In [6]:
restaurants = pd.read_sql("SELECT * FROM restaurants", connector)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
merged_df = orders.merge(users,
                         on="user_id",
                         how="left")

In [8]:
final_df = merged_df.merge(restaurants,
                          on="restaurant_id",
                          how="left")


In [9]:
final_df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")
df['order_date'] = pd.to_datetime(df['order_date'])


C:\Users\Danish\AppData\Local\Temp\ipykernel_12556\106848794.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


In [ ]:
df[df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)


city
Chennai    1080909.79
Name: total_amount, dtype: float64

In [ ]:
df.groupby('cuisine')['total_amount'] \
  .mean() \
  .sort_values(ascending=False) \
  .head(1)


cuisine
Mexican    808.021344
Name: total_amount, dtype: float64

In [ ]:
high_spenders = df.groupby('user_id')['total_amount'].sum()
(high_spenders > 1000).sum()
high_spenders.describe()



count     2883.000000
mean      2778.919223
std       1627.276076
min        102.220000
25%       1563.495000
50%       2514.920000
75%       3715.145000
max      11556.490000
Name: total_amount, dtype: float64

In [ ]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5','3.6–4.0','4.1–4.5','4.6–5.0']

df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels)

df.groupby('rating_range')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


C:\Users\Danish\AppData\Local\Temp\ipykernel_12556\2805884943.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('rating_range')['total_amount'] \


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [ ]:
df[df['membership'] == 'Gold'] \
  .groupby('city')['total_amount'] \
  .mean() \
  .sort_values(ascending=False) \
  .head(1)


city
Chennai    808.45908
Name: total_amount, dtype: float64

In [ ]:
restaurant_counts = df.groupby('cuisine')['restaurant_id'].nunique()
revenue = df.groupby('cuisine')['total_amount'].sum()

pd.concat([restaurant_counts, revenue], axis=1) \
  .rename(columns={'restaurant_id':'num_restaurants'}) \
  .sort_values('num_restaurants')


,num_restaurants,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [ ]:
gold_pct = (df[df['membership']=='Gold'].shape[0] / df.shape[0]) * 100
round(gold_pct)


50

In [ ]:
restaurant_stats = df.groupby('restaurant_name').agg(
    avg_order_value=('total_amount','mean'),
    total_orders=('order_id','count')
)

restaurant_stats[restaurant_stats['total_orders'] < 20] \
    .sort_values('avg_order_value', ascending=False) \
    .head(1)


KeyError: 'restaurant_name'

In [ ]:
df.groupby(['membership','cuisine'])['total_amount'] \
  .sum() \
  .sort_values(ascending=False) \
  .head(1)


membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64

In [ ]:
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [ ]:
df['month'] = df['order_date'].dt.month
df['quarter'] = df['order_date'].dt.quarter
df['day'] = df['order_date'].dt.day_name()
